In [1]:
import utils
import gaussian_cal
import pandas as pd
import subprocess
import os
import time
from utils import i8cpu_running
name_list = ['S','A','EHOPA','AEPYridine','AEP','APN','DBE','DIPEDA','OA','mXD']
smiles_list = [
'FC1(C(F)(C(F)(F)F)F)C(OC(F)(F)C(F)1F)(C(F)(C(F)(F)F)F)F',
'FC1(C(F)(C(F)(F)F)F)C(OC(F)(F)C(F)1F)(C(F)(C(O)=O)F)F',
'CCCCC(CC)COCCCNC(C(F)(F)C1(F)OC(F)(F)C(F)(F)C1(C(F)(C(O)=O)F)F)=O',
'FC1(F)C(F)(F)C(F)(C(F)(C(O)=O)F)C(F)(C(F)(C(NCCC2=CN=CC=C2)=O)F)O1',
'FC1(F)C(F)(F)C(F)(C(F)(C(O)=O)F)C(F)(C(F)(C(NCCN2CCCCC2)=O)F)O1',
'FC1(F)C(F)(F)C(F)(C(F)(C(O)=O)F)C(F)(C(F)(C(NC2=CC(C#N)=CC(C#N)=C2)=O)F)O1',
'FC1(F)C(F)(F)C(F)(C(F)(C(O)=O)F)C(F)(C(F)(C(NCCN(CCCC)CCCC)=O)F)O1',
'FC1(F)C(F)(F)C(F)(C(F)(C(O)=O)F)C(F)(C(F)(C(NCCN(C(C)C)C(C)C)=O)F)O1',
'FC1(F)C(F)(F)C(F)(C(F)(C(O)=O)F)C(F)(C(F)(C(NCCCCCCCC)=O)F)O1',
'FC1(F)C(F)(F)C(F)(C(F)(C(O)=O)F)C(F)(C(F)(C(NCC2=CC(CN)=CC=C2)=O)F)O1',
]


In [2]:
for _ in name_list:
    try:
        os.system(f"cp -r CYTOP/{_}/* .")
    except:
        pass


In [3]:
G0 = gaussian_cal.GaussianCal(method='cam-B3LYP',basis='6-311G(d,p)',charge='pos',wfn=True,debug=True)
G0.log()

for i in range(len(name_list)):
    for j in range(200):
        if os.path.exists(f"{name_list[i]}{j}"):
            # if not utils.xyzcheck(f'{name_list[i]}{j}_0.xyz',smiles_list[i]):
            #     print(f"{name_list[i]}{j}_0.xyz is not correct")
            #     continue
            while utils.i8cpu_running():
                time.sleep(60)
            G0.Run(f"{name_list[i]}{j}_0.xyz")

Submitted batch job 1693097
Submitted batch job 1693100


KeyboardInterrupt: 

In [29]:
output = subprocess.check_output("squeue", shell=True, text=True)
output.count('i8cpu')


int

In [31]:
G0.Run("APN0_0.xyz")

Submitted batch job 1693091


In [32]:
def time_calculation(log):
    with open(log, 'r') as file:
        for line in file:
            if "Job cpu time" in line:
                cpu_time = line.split(":")[1].strip()  # 获取冒号后面的部分并去除首尾空格
                print(f"Job cpu time: {cpu_time}")
                break

In [36]:
time_calculation("APN0/APN0_p1.log")

Job cpu time: 0 days  1 hours  7 minutes 45.2 seconds.
